1. What are the 1st most common and the 2nd most common nested relations *and* 
2. What is their depth (distance from [N] of the main relation)?

Check the following:
- Attribution
- Explanation
- Contrast
- Background
- Evaluation
- Condition
- Topic-comment



Example of the Result: 
{Attribution [N]: 
{Background: [[depth 1, 70 times], [depth 2, 45 times]]}, 
{Explanation: [[depth 2, 50 times], [depth 5, 15 times]]}}

In [23]:
from collections import defaultdict
from functools import reduce

In [2]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [3]:
%run ./utils.ipynb
%run ./relation_extraction.ipynb

In [33]:
def count(file_name, relations, rel_counts):
    for rel in relations:
        if rel is not None:
            if rel.left.type == "S" and rel.left_child:
                child = rel.left_child
            elif rel.right.type == "S" and rel.right_child:
                child = rel.right_child
            else:
                continue
            rel_counts[
                (file_name, rel.type, child.type)
            ] += 1

In [46]:
def counts_to_table(rel_counts):
    return pd.DataFrame(
        [key + (cnt,) for key, cnt in dict(rel_counts).items()],
        columns=["file_name", "relation", "nested", "count"]
    )

In [52]:
DIRECTORY = "parsed/race/train/middle"

def count_relation_pairs():
    rel_counts = defaultdict(int)
    for file_name in os.listdir(DIRECTORY):
        path = os.path.join(DIRECTORY, file_name)
        _, relation_map = extract_relations(path)
        relations = reduce(
            lambda a, b: a + b,
            list(relation_map.values())
        )
        count(file_name, relations, rel_counts)
    return counts_to_table(rel_counts)

cnt_df = count_relation_pairs()

In [68]:
total_texts = cnt_df.file_name.nunique()

In [76]:
share_df = cnt_df.drop(["file_name"], axis=1)\
      .groupby(["relation", "nested"]).count() / total_texts * 100

In [86]:
unstacked = share_df.unstack("nested").fillna(0)
unstacked = unstacked.iloc[np.argsort(-unstacked.max(axis=1).values)]
unstacked.columns = unstacked.columns.droplevel()
unstacked.columns.name = "nested_relation"

In [108]:
-np.array(sorted(-unstacked.values.ravel()))[:5]

array([96.38932496, 68.86970173, 54.0188383 , 50.04709576, 49.90580848])

In [115]:
print("PERCENTAGE OF TEXTS WITH A GIVEN RELATION-SATTELITE PAIR")
display(unstacked)

PERCENTAGE OF TEXTS WITH A GIVEN RELATION-SATTELITE PAIR


nested_relation,Attribution,Background,Cause,Comparison,Condition,Contrast,Elaboration,Enablement,Evaluation,Explanation,Joint,Manner-Means,Summary,Temporal,Topic-Comment,same-unit,textual-organization
relation,,,,,,,,,,,,,,,,,
Elaboration,50.047096,18.587127,4.301413,1.287284,12.276295,39.905808,96.389325,10.894819,14.772370,49.905808,68.869702,1.993721,0.769231,12.621664,7.990581,19.497645,0.502355
Explanation,8.477237,3.736264,1.161695,0.188383,2.543171,5.196232,54.018838,0.643642,3.642072,10.596546,17.974882,0.219780,0.015699,2.967033,0.769231,2.339089,0.062794
Contrast,6.326531,3.029827,0.502355,0.078493,0.518053,1.255887,13.014129,1.130298,0.361068,3.814757,2.778650,0.156986,0.015699,0.345369,0.031397,1.946625,0.015699
Evaluation,3.861852,0.518053,0.125589,0.031397,0.266876,4.034537,10.894819,0.361068,0.518053,8.210361,2.087912,0.062794,0.000000,0.188383,0.109890,0.690738,0.000000
Background,3.375196,2.684458,0.235479,0.078493,0.345369,0.737834,9.576138,0.784929,0.172684,0.753532,3.830455,0.062794,0.000000,1.522763,0.031397,1.334380,0.000000
Attribution,4.474097,0.533752,0.109890,0.047096,0.141287,0.125589,5.023548,0.188383,0.000000,0.078493,3.956044,0.062794,0.015699,0.376766,0.000000,3.061224,0.000000
Enablement,1.302983,0.266876,0.015699,0.031397,0.078493,0.031397,3.296703,0.533752,0.000000,0.031397,1.742543,0.156986,0.000000,0.141287,0.000000,0.141287,0.000000
Condition,2.417582,0.518053,0.031397,0.047096,2.558870,0.109890,2.433281,0.282575,0.031397,0.047096,1.930926,0.094192,0.000000,0.219780,0.000000,0.596546,0.000000
Summary,0.078493,0.000000,0.015699,0.031397,0.156986,0.109890,2.182104,0.031397,0.094192,0.533752,0.047096,0.000000,0.000000,0.015699,0.031397,0.015699,0.000000


In [100]:
unstacked.to_excel("relation_pair_text_percentages.xlsx")